In [92]:
import numpy as np

class FeedForwardNetwork:
    
    def __init__(self):
        self.synapticWeights = []
        self.optimizer = None
    
    def add(self, units, input_shape=None):
        if not self.synapticWeights:
            if input_shape:
                self.synapticWeights.append(np.random.rand(input_shape, units))
            return
        else:
            self.synapticWeights.append(np.random.rand(self.synapticWeights[-1].shape[1], units))
    
    def compile(self, optimizer='sgd'):
        self.optimizer = optimizer
    
    def sigmoid(self, X, deriv=False):
        if not deriv:
            return 1/(1+np.exp(-X))
        return X*(1-X)
    
    def fit(self, X, y, batch_size=32,epochs=10, learning_rate=0.1):
        
        layers = len(self.synapticWeights)
        forward = [0]*layers
        backward = [0]*layers
        if self.optimizer =='momentum':
            v = [[0]*layers for i in range(epochs)]
        
        for k in range(epochs):
            for i in range(layers):
                if i==0:
                    forward[i] = self.sigmoid(X.dot(self.synapticWeights[i]))
                else:
                    forward[i] = self.sigmoid(forward[i-1].dot(self.synapticWeights[i]))

            for i in range(layers-1, -1, -1):

                if i==layers-1:
                    error = forward[i] - y
                    backward[i] = error*self.sigmoid(forward[i], deriv=True)/2
                else:
                    error = backward[i+1].dot(self.synapticWeights[i+1].T)
                    backward[i] = error*self.sigmoid(forward[i], deriv=True)/2
            
            if self.optimizer == 'sgd':
                for i in range(layers):
                    if i==0:
                        self.synapticWeights[i] = self.synapticWeights[i] - X.T.dot(backward[i]) * learning_rate
                    else:
                        self.synapticWeights[i] = self.synapticWeights[i] - forward[i-1].T.dot(backward[i]) * learning_rate
                        
            elif self.optimizer == 'momentum':

                for i in range(layers):
                    if k==0:
                        if i==0:
                            v[k][i] = X.T.dot(backward[i]) * learning_rate
                            self.synapticWeights[i] = self.synapticWeights[i] - v[k][i]
                        else:
                            v[k][i] = forward[i-1].T.dot(backward[i]) * learning_rate
                            self.synapticWeights[i] = self.synapticWeights[i] - v[k][i]
                    else:
                        if i==0:
                            v[k][i] = 0.99*v[k-1][i] + X.T.dot(backward[i]) * learning_rate
                            self.synapticWeights[i] = self.synapticWeights[i] - v[k][i]
                        else:
                            v[k][i] = 0.99*v[k-1][i] + forward[i-1].T.dot(backward[i]) * learning_rate
                            self.synapticWeights[i] = self.synapticWeights[i] - v[k][i]
                        
    
    def predict(self, X):
        layers = len(self.synapticWeights)
        forward = [0]*layers
        for i in range(layers):
                if i==0:
                    forward[i] = self.sigmoid(np.array(X).dot(self.synapticWeights[i]))
                else:
                    forward[i] = self.sigmoid(forward[i-1].dot(self.synapticWeights[i]))
        return forward[layers-1]
    

In [107]:
X = np.array([[0, 0], [1, 0], [0, 1], [1,1]])
y = np.array([[0], [1], [1], [0]])

In [116]:
model = FeedForwardNetwork()
model.add(16, input_shape=2)
model.add(1)

In [117]:
model.compile('sgd')
model.fit(X, y,32,50000, 0.1)

In [118]:
model.predict([[0, 0], [1, 0], [0, 1], [1,1]])

array([[0.04738408],
       [0.95122423],
       [0.95068052],
       [0.05017581]])

In [119]:
model2 = FeedForwardNetwork()
model2.add(16, input_shape=2)
model2.add(1)

In [121]:
model2.compile('momentum')
model2.fit(X, y,32,50000, 0.1)

In [122]:
model2.predict([[0, 0], [1, 0], [0, 1], [1,1]])

array([[0.00240173],
       [0.99740965],
       [0.9973635 ],
       [0.00278563]])